In [1]:
import os
os.chdir(r'/workspaces/expense_manager')
print(f'current working directory set to : {os.getcwd()}')

current working directory set to : /workspaces/expense_manager


In [2]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


/workspaces/expense_manager/.venv/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [3]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1YLVz3i6R9sYXW3oWfUEtFTMkrXyY6ONB4FohhTf4B8w"
SAMPLE_RANGE_NAME = "taxonomy!A1:C10"

In [12]:
def main():
  """Shows basic usage of the Sheets API.
  Prints values from a sample spreadsheet.
  """
  creds = None
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    print('Calling sheets API')

    spreadsheet = service.spreadsheets().get(spreadsheetId=SAMPLE_SPREADSHEET_ID).execute()

    sheet_name = spreadsheet["properties"]["title"]

    print("Spreadsheet name:", sheet_name)
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
        .execute()
    )
    print(result)
    values = result.get("values", [])

    if not values:
      print("No data found.")
      return

    print("Name, Major:")
    for row in values:
      # Print columns A and E, which correspond to indices 0 and 4.
      print(f"{row[0]}, {row[1]}")
  except HttpError as err:
    print(err)


main()

Calling sheets API
Spreadsheet name: Taxonomy Updated
{'range': 'taxonomy!A1:C10', 'majorDimension': 'ROWS', 'values': [['Category', 'Sub Category-I', 'Sub Category II'], ['Food Items', 'Fruits and Vegetables', 'Fruits'], ['Food Items', 'Fruits and Vegetables', 'Fruits'], ['Food Items', 'Fruits and Vegetables', 'Fruits'], ['Food Items', 'Fruits and Vegetables', 'Vegetables'], ['Food Items', 'Fruits and Vegetables', 'Vegetables'], ['Food Items', 'Fruits and Vegetables', 'Vegetables'], ['Food Items', 'Fruits and Vegetables', 'Vegetables'], ['Food Items', 'Fruits and Vegetables', 'Vegetables'], ['Food Items', 'Fruits and Vegetables', 'Vegetables']]}
Name, Major:
Category, Sub Category-I
Food Items, Fruits and Vegetables
Food Items, Fruits and Vegetables
Food Items, Fruits and Vegetables
Food Items, Fruits and Vegetables
Food Items, Fruits and Vegetables
Food Items, Fruits and Vegetables
Food Items, Fruits and Vegetables
Food Items, Fruits and Vegetables
Food Items, Fruits and Vegetables


In [4]:
from src.integration.gsheet_handler import GSheetHandler

SPREADSHEET_ID = "1YLVz3i6R9sYXW3oWfUEtFTMkrXyY6ONB4FohhTf4B8w"
cred = 'credentials.json'
gsheet_handler = GSheetHandler(sheet_id=SPREADSHEET_ID, credential_file=cred)

2025-12-10 05:00:15,447 - src.integration.gsheet_handler - INFO - [60] - Successfully authenticated and loaded Google Sheet.


In [14]:
df = gsheet_handler.load_sheet_as_df(worksheet_name='taxonomy')
df = df.iloc[:, :3]
df.rename(columns={
    'Category':'category',
    'Sub Category-I':'sub_category_i',
    'Sub Category II':'sub_category_ii',
}, inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)

2025-12-10 05:18:53,690 - src.integration.gsheet_handler - INFO - [83] - Loaded worksheet 'taxonomy' into DataFrame.
/tmp/ipykernel_5193/920912373.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={
/tmp/ipykernel_5193/920912373.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [15]:
df

,category,sub_category_i,sub_category_ii
0,Food Items,Fruits and Vegetables,Fruits
1,Food Items,Fruits and Vegetables,Vegetables
2,Food Items,"Grains, Flours & Pulses",Rice
3,Food Items,"Grains, Flours & Pulses",Flours
4,Food Items,"Grains, Flours & Pulses",Oats & Cereals
...,...,...,...
168,Utilities and Bills,Household Bills,Utilities
169,Utilities and Bills,Communication,Connectivity
170,Utilities and Bills,Subscriptions,Digital Services
171,Utilities and Bills,Housing,Accommodation


In [ ]:
df